# Проектирование нашего эксперимента

Что текущий коэффициент конверсии составляет в среднем около 13% в течение года, и что команда будет довольна увеличением на 2%, а это означает, что новый дизайн будет считаться успешным, если он повысит коэффициент конверсии до 15%.

1) ГИПОТЕЗА:  внедрив новый дизайн страницы, мы увеличим конверсию в покупку на  2%.

Учитывая, что мы не знаем, будет ли новый дизайн работать лучше или хуже (или так же?), как наш текущий дизайн, мы выберем двусторонний тест:
Hₒ: p = pₒ
Hₐ: p ≠ pₒ

α = 0.05
β = 0.8

2) Для нашего теста нам понадобятся две группы:
Группа  control - им будет показан старый дизайн

Группа treatment (или тестовая) - им будет показан новый дизайн


3) МЕТРИКА: коэффициент конверсии

В данных - conversion rate:

0 - Пользователь не покупал продукт во время этой пользовательской сессии

1 - Пользователь купил продукт во время этой пользовательской сессии

4) ЕДИНИЦА РАНДОМИЗАЦИИ: пользователи


# Сбор и подготовка данных

In [84]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.api as sms
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil

effect_size = sms.proportion_effectsize(0.13, 0.15)    #вычисляем cohen's d (размер эффекта)

required_n = sms.NormalIndPower().solve_power(         #вычисляем обьем выборок
    effect_size, 
    power=0.8, 
    alpha=0.05, 
    ratio=1
    )                                                 

required_n = ceil(required_n)                                                    

print("Объем выборки = ", required_n)

Объем выборки =  4720


In [33]:
df = pd.read_csv('downloads/ab_data.csv')  #считываем данные

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294478 non-null  int64 
 1   timestamp     294478 non-null  object
 2   group         294478 non-null  object
 3   landing_page  294478 non-null  object
 4   converted     294478 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


In [87]:
df['count'] = df['user_id'].map(df['user_id'].value_counts())  #выводим сколько раз встречается в таблице каждый user_id
df = df.drop(df[df['count'] > 1].index) #удаляем user_id, которые встречаются больше 1-ого раза


In [135]:
#создаем две рандомных выборки с помощью sample (выборки контроля и теста)
df_control_sample = df[df["group"] == 'control'].sample(n = required_n, replace=True)
df_treatment_sample = df[df["group"] == 'treatment'].sample(n = required_n , replace=True)

# Проверка гипотезы

Базовая статистика для контрольной выборки:

In [136]:
std_control = np.std(df_control_sample['converted'], ddof=0)   #Стандартное отклонение
se_control = stats.sem(df_control_sample['converted'], ddof=0)  #Стандартная ошибка
conversion_rate_control = df_control_sample['converted'].sum() / required_n   #Коэффициент конверсии
print('Стандартное отклонение =', round(std_control, 3),',','Стандартная ошибка =', round(se_control, 3),',', 'Коэффициент конверсии =', round(conversion_rate_control, 3))


Стандартное отклонение = 0.321 , Стандартная ошибка = 0.005 , Коэффициент конверсии = 0.117


Базовая статистика для тестовой выборки:

In [137]:
std_treatment = np.std(df_treatment_sample['converted'], ddof=0)   #Стандартное отклонение
se_treatment = stats.sem(df_treatment_sample['converted'], ddof=0)    #Стандартная ошибка
conversion_rate_treatment = df_treatment_sample['converted'].sum() / required_n    #Коэффициент конверсии
print('Стандартное отклонение =', round(std_treatment, 3),',','Стандартная ошибка =', round(se_treatment, 3),',', 'Коэффициент конверсии =', round(conversion_rate_treatment, 3))


Стандартное отклонение = 0.328 , Стандартная ошибка = 0.005 , Коэффициент конверсии = 0.123


Конверсия в тестовой группе немного выше, является ли эта разница статистически значимой?

Для показателей конверсии используем z-тест для двух пропорций:

In [138]:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint


# Данные для первой выборки
successes1 = conversion_rate_control * required_n
nobs1 = required_n

# Данные для второй выборки
successes2 = conversion_rate_treatment * required_n
nobs2 = required_n

# Выполнение z-теста
count = [successes1, successes2]
nobs = [nobs1, nobs2]
stat, pval = proportions_ztest(count, nobs)

# Вычисление доверительного интервала
ci_low, ci_upp = proportion_confint(successes2, nobs2)

# Вывод результатов
print('Доверительный интервал для treatment: ({:.3f}, {:.3f})'.format(ci_low, ci_upp))

# Вывод результатов
print('stat=%.3f, p=%.3f' % (stat, pval))
if pval > 0.05:
    print('Пропорции не имеют статистически значимые различия')
else:
    print('Пропорции имеют статистически значимые различия')



Доверительный интервал для treatment: (0.113, 0.132)
stat=-0.888, p=0.375
Пропорции не имеют статистически значимые различия


# Выводы

Поскольку наше p-значение намного выше нашего порога α = 0,05, мы не можем отвергнуть нулевую гипотезу H-, что означает, что наш новый дизайн не работал значительно иначе, чем наш старый.

Кроме того, если мы посмотрим на доверительный интервал для группы treatment (0.113, 0.132) или 11,3-13,2%), мы заметим, что:

Он включает в себя наше базовое значение коэффициента конверсии 13%
Он не включает наше целевое значение в 15% (рост на 2%, к которому мы стремились)
Это означает, что более вероятно, что истинный коэффициент конверсии нового дизайна аналогичен нашему базовому уровню, а не целевому показателю в 15%, на который мы надеялись. 